In [49]:
import pandas as pd
import numpy as np
import re
from pandas import DataFrame,Series
from datetime import datetime

# 数据读入

In [50]:
data = pd.read_table("xiecheng_hotel.txt",error_bad_lines=False)
data.shape

(84890, 10)

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84890 entries, 0 to 84889
Data columns (total 10 columns):
No      84890 non-null object
数据源     84890 non-null object
酒店      84890 non-null object
地段      84890 non-null object
用户ID    84890 non-null object
评分      55499 non-null float64
住宿类型    55555 non-null object
入住时间    55484 non-null object
入住房型    55497 non-null object
评论内容    83838 non-null object
dtypes: float64(1), object(9)
memory usage: 6.5+ MB


In [52]:
data.head()

,No,数据源,酒店,地段,用户ID,评分,住宿类型,入住时间,入住房型,评论内容
0,C000001,携程,7天,步行街,_CFT0229991****,4.0,朋友出游,2017年08月入住,自主大床房,位置可以，距离地铁站近，离步行街也挺近的，房间很小，没有窗户，不太通风，价格贵，可以免费寄存...
1,C000002,携程,7天,步行街,110153****,3.5,家庭亲子,2017年07月入住,自主双床房,酒店位置不错，就在步行街，出门吃饭，搭地铁很方便。只是房间太小，设施简陋了些，不过为了方便和...
2,C000003,携程,7天,步行街,M28432****,1.0,朋友出游,2017年08月入住,自主大床房,太太太垃圾了，7天酒店的形象在我心中彻底破坏，从没见过如此差劲的酒店，洗水间头上漏水，边洗澡...
3,C000004,携程,7天,步行街,M10416****,4.8,家庭亲子,2017年08月入住,自主双床房,周围环境一般的，主要是在步行街上，去南京路方便了，地铁二号线乘坐也方便，贵州路上有1个安徽帅...
4,C000005,携程,7天,步行街,M222136****,3.5,情侣出游,2017年06月入住,自主大床房,NaN


In [53]:
data.columns

Index(['No', '数据源', '酒店', '地段', '用户ID', '评分', '住宿类型', '入住时间', '入住房型', '评论内容'], dtype='object')

# 去掉无用字段

In [54]:
data.drop(['No','数据源'],axis = 1,inplace = True)

# 去掉同人重复评论

In [55]:
data.drop_duplicates(subset = ['用户ID','评论内容'],keep = 'first',inplace = True)

In [56]:
data.shape

(83125, 8)

# 缺失值、异常值观察

In [57]:
cols = list(data.columns)
missing = list(map(lambda x:data[x].isnull().sum(),cols))
missing_dict = dict(zip(cols,missing))
missing_dict

{'住宿类型': 29066,
 '入住房型': 29124,
 '入住时间': 29137,
 '地段': 0,
 '用户ID': 0,
 '评分': 29122,
 '评论内容': 960,
 '酒店': 0}

## 评论内容

In [58]:
cond = data['评论内容'].isnull()
data = data[~cond]
data.shape

(82165, 8)

### 筛掉网页标识符

In [59]:
cond = data['评论内容'].str.contains(r'<b></b>')
data.loc[cond]

,酒店,地段,用户ID,评分,住宿类型,入住时间,入住房型,评论内容


### 筛掉非中文字符串

In [60]:
cond = data['评论内容'].str.contains(r'[一-龥]')
data.loc[~cond]

,酒店,地段,用户ID,评分,住宿类型,入住时间,入住房型,评论内容
277,7天,步行街,E37661****,3.3,代人预订,2017年05月入住,自主大床房,"Almost on Nanjing Road Pedestrian Street, that..."
463,7天,步行街,_FB50891****,3.3,家庭亲子,2017年01月入住,自主大床房,하루밤 그냥 묵기는 괜찮아요!!!!!!!!!!!!!
601,7天,步行街,E17455****,3.0,商务出差,2017年02月入住,自主双床房,"Hotel bien ubicado, muy económico, buena habit..."
631,7天,步行街,D35324****,3.8,商务出差,2016年04月入住,自主双床房,It is located in the central area of shopping ...
845,7天,步行街,E28377****,4.3,独自旅行,2016年06月入住,自主大床房,위치는 정말 좋습니다. 위치는 정말좋습니다. \n위치는 정말 좋습니다. 위치는 정말...
1238,7天,步行街,D30608****,3.0,独自旅行,2016年02月入住,经济房,"You get what you pay for, or so they say."
1920,7天,步行街,_WeChat56261****,5.0,朋友出游,2017年04月入住,自主大床房,good
1972,7天,步行街,santoshsoni****,1.8,家庭亲子,2016年12月入住,商务双床房,Only for enjoyment coming this hotel other mat...
2025,7天,步行街,M39862****,5.0,朋友出游,2016年07月入住,自主双床房,ok
2035,7天,步行街,E1118****,1.5,朋友出游,2016年06月入住,自主大床房,"not so coo, it was so so and for emmergency pe..."


In [61]:
data = data.loc[cond]
data.shape

(81496, 8)

### 筛掉刷评论数据

In [62]:
cond = data['评论内容'].str.contains(r'酒店位置好找，交通方便')
data.loc[cond]

,酒店,地段,用户ID,评分,住宿类型,入住时间,入住房型,评论内容
9680,汉庭,人名广场,_CFT010000000117****,5.0,商务出差,2015年07月入住,高级大床房,酒店位置好找，交通方便。房间宽敞明亮，干净整洁，住着安心。性价比超高，酒店很好，前台服务也很...
11405,汉庭,人名广场,1352481****,5.0,商务出差,2014年11月入住,高级大床房,酒店位置好找，交通方便。房间宽敞明亮，干净整洁，住着安心。性价比超高，酒店很好，前台服务也很...
11420,汉庭,人名广场,210529****,5.0,商务出差,2014年11月入住,高级大床房,酒店位置好找，交通方便。房间宽敞明亮，干净整洁，住着安心。性价比超高，酒店很好，前台服务也很...
12385,汉庭,人名广场,艺龙网用户,NaN,NaN,NaN,NaN,酒店位置好找，交通方便，性价比超高，酒店很好，前台服务也很好，客房阿姨热情，整个酒店都有免费...


In [63]:
data = data.loc[~cond]

In [64]:
data.shape

(81492, 8)

In [65]:
data['评论内容'] = data['评论内容'].str.strip("\n")

In [66]:
data['评论内容'] = data['评论内容'].str.strip("\r")

In [67]:
data['评论内容'] = data['评论内容'].str.replace("\n","")

## 酒店

In [68]:
data['酒店'].value_counts()

全季       20567
如家       16516
莫泰168    13942
锦江之星     12095
汉庭        9486
7天        8886
Name: 酒店, dtype: int64

## 地段

In [69]:
data['地段'].value_counts()

步行街     17084
虹桥      11782
会展中心     9346
延安路      7137
徐家汇      5007
城隍庙      3933
武宁路      3805
火车站      3381
川沙       3269
福州路      3237
外白渡桥     3218
人名广场     3206
外滩       3062
漕溪路      2548
羽山路      1477
Name: 地段, dtype: int64

In [70]:
data['地段'].replace('人名广场','人民广场',inplace = True)

## 评分

In [71]:
data['评分'].dtypes

dtype('float64')

In [72]:
data.groupby('酒店')['评分'].agg(np.mean)

酒店
7天       4.047010
全季       4.602427
如家       4.385482
汉庭       4.341235
莫泰168    4.568979
锦江之星     4.543984
Name: 评分, dtype: float64

## 住宿类型

In [73]:
data['住宿类型'].value_counts()

商务出差    19862
家庭亲子     9852
朋友出游     8806
情侣出游     6266
独自旅行     3734
其它       2252
代人预订     1695
Name: 住宿类型, dtype: int64

In [74]:
data['住宿类型'].fillna("其它",inplace = True)

In [75]:
data['住宿类型'].value_counts()

其它      31277
商务出差    19862
家庭亲子     9852
朋友出游     8806
情侣出游     6266
独自旅行     3734
代人预订     1695
Name: 住宿类型, dtype: int64

## 入住时间

In [76]:
data['入住时间'] = data['入住时间'].str.replace("入住","")
data['入住时间'].value_counts()

2017年04月    2404
2017年06月    2355
2017年05月    2245
2017年07月    2084
2017年03月    2015
2015年10月    1867
2016年08月    1865
2016年10月    1783
2017年08月    1694
2016年07月    1601
2016年09月    1592
2017年02月    1549
2015年04月    1547
2016年06月    1448
2016年11月    1447
2015年11月    1446
2015年07月    1414
2017年01月    1397
2016年12月    1394
2015年08月    1378
2015年06月    1356
2015年03月    1342
2016年05月    1338
2015年09月    1331
2015年05月    1256
2016年04月    1228
2015年01月    1186
2015年12月    1150
2015年02月    1128
2016年01月    1013
2016年02月     997
2016年03月     991
2014年12月     896
2014年10月     768
2014年11月     719
2014年09月     690
2014年08月     346
2014年07月      99
2014年06月      34
2014年05月       2
2014年04月       1
Name: 入住时间, dtype: int64

## 入住房型

In [77]:
data['入住房型'].value_counts()

高级大床房        7589
豪华大床房        3924
大床房          3646
商务大床房        3566
双床房          3098
特惠大床房        2721
自主大床房        2387
标准双人房        2211
商务房A         2104
特惠双人房        2084
豪华双床房        1824
标准房A         1813
套房           1124
零压-高级大床房      992
经济房           926
自主双床房         904
精选商务房B        865
家庭房           828
景观大床房         786
商务房B          749
高级大床房A        726
大床房(无窗)       565
特价房           489
精选标准房A        480
任意房型          470
特惠标准房(无窗)     425
特惠商务房(无窗)     403
标准双床房         376
特惠大床房(无窗)     365
零压-双床房        329
             ... 
商务房C           22
特惠房            20
标准房B(无窗)       18
精选单人房C         18
促销双床房          15
时尚双人房          13
标准房C           12
精选双床间          12
标准套房A          11
商务套房A          11
单人房B           10
特惠豪华大床房         8
时租房4小时          8
精选单人房           5
特惠商务房           3
特惠双床房           2
双床间             2
豪华双人间           2
豪华套房            2
高级大床房/双床房       2
特惠豪华双床房         2
豪华特大号床间         2
单人房B（无窗）        1
商务房C（无窗）        1
日式大号床间    

In [78]:
cols = list(data.columns)
missing = list(map(lambda x:data[x].isnull().sum(),cols))
missing_dict = dict(zip(cols,missing))
missing_dict

{'住宿类型': 0,
 '入住房型': 29081,
 '入住时间': 29096,
 '地段': 0,
 '用户ID': 0,
 '评分': 29081,
 '评论内容': 0,
 '酒店': 0}

In [79]:
data.head()

,酒店,地段,用户ID,评分,住宿类型,入住时间,入住房型,评论内容
0,7天,步行街,_CFT0229991****,4.0,朋友出游,2017年08月,自主大床房,位置可以，距离地铁站近，离步行街也挺近的，房间很小，没有窗户，不太通风，价格贵，可以免费寄存...
1,7天,步行街,110153****,3.5,家庭亲子,2017年07月,自主双床房,酒店位置不错，就在步行街，出门吃饭，搭地铁很方便。只是房间太小，设施简陋了些，不过为了方便和...
2,7天,步行街,M28432****,1.0,朋友出游,2017年08月,自主大床房,太太太垃圾了，7天酒店的形象在我心中彻底破坏，从没见过如此差劲的酒店，洗水间头上漏水，边洗澡...
3,7天,步行街,M10416****,4.8,家庭亲子,2017年08月,自主双床房,周围环境一般的，主要是在步行街上，去南京路方便了，地铁二号线乘坐也方便，贵州路上有1个安徽帅...
5,7天,步行街,1500108****,3.3,朋友出游,2017年07月,自主大床房,位置可以，就是酒店服务太垃圾，前台告知只有电卡没有房卡，开房门得服务员跟着，还让交200押金...


In [80]:
data.shape

(81492, 8)

In [81]:
data.index = range(0,len(data))

In [82]:
data.to_csv("clean_xiecheng_0918.txt",sep = "\t",encoding = "utf-8",index = False)

# 关键词筛选

In [83]:
words = []
with open("keywords.txt","r",encoding='utf-8') as f:
    for word in f.readlines():
        words.append(word.strip().strip("\ufeff"))

In [84]:
words

['全季',
 '如家',
 '莫泰168',
 '锦江之星',
 '汉庭',
 '7天',
 '快捷酒店',
 '房间',
 '设施',
 '齐全',
 '空调',
 '电话',
 '大床',
 '窗户',
 '环境',
 '装修',
 '隔音',
 '吵',
 '声音',
 '前台',
 '卫生间',
 '方便',
 '交通',
 '步行',
 '走路',
 '位置',
 '好找',
 '停车',
 '外滩',
 '步行街',
 '城隍庙',
 '南京路',
 '机场',
 '火车站',
 '地铁站',
 '高架',
 '服务',
 '服务员',
 '人员',
 '态度',
 '预定',
 '入住',
 '退房',
 '吃饭',
 '早餐',
 '干净',
 '舒服',
 '舒适',
 '安静',
 '温馨',
 '整洁',
 '价格',
 '价位',
 '实惠',
 '性价比',
 '免费',
 '早上',
 '晚上',
 '出差',
 '商务',
 '度假',
 '朋友']

In [85]:
data['关键词'] = data['评论内容'].copy().str.findall('|'.join(words))

In [86]:
data['关键词'] = data['关键词'].map(set).map(list)

In [87]:
data.tail()

,酒店,地段,用户ID,评分,住宿类型,入住时间,入住房型,评论内容,关键词
81487,如家,会展中心,jer****,3.0,商务出差,2015年03月,特惠大床房,还可以,[]
81488,如家,会展中心,M51856****,5.0,商务出差,2017年05月,特惠大床房,位置挺好的,[位置]
81489,如家,会展中心,E4993****,4.3,商务出差,2017年05月,特惠双人房,交通便利、,[交通]
81490,如家,会展中心,童-Alina,5.0,其它,2017年06月,特惠双人房,真的超级好，特别满意，很干净，客服态度也很好，收拾房间的阿姨也很贴心，交通还算便利，而且没有...,"[交通, 干净, 态度, 房间, 舒服]"
81491,如家,会展中心,中国潇洒的男人,3.3,家庭亲子,2014年09月,标准双人房,一般,[]


In [88]:
data.shape

(81492, 9)

In [89]:
cond = data['关键词'].map(len) > 0
data = data.loc[cond]
data.shape

(62873, 9)

In [90]:
data.index = range(len(data))
data.head(10)

,酒店,地段,用户ID,评分,住宿类型,入住时间,入住房型,评论内容,关键词
0,7天,步行街,_CFT0229991****,4.0,朋友出游,2017年08月,自主大床房,位置可以，距离地铁站近，离步行街也挺近的，房间很小，没有窗户，不太通风，价格贵，可以免费寄存...,"[位置, 价格, 地铁站, 免费, 窗户, 步行, 房间]"
1,7天,步行街,110153****,3.5,家庭亲子,2017年07月,自主双床房,酒店位置不错，就在步行街，出门吃饭，搭地铁很方便。只是房间太小，设施简陋了些，不过为了方便和...,"[方便, 位置, 设施, 吃饭, 步行, 房间]"
2,7天,步行街,M28432****,1.0,朋友出游,2017年08月,自主大床房,太太太垃圾了，7天酒店的形象在我心中彻底破坏，从没见过如此差劲的酒店，洗水间头上漏水，边洗澡...,"[7天, 空调, 免费, 如家, 房间]"
3,7天,步行街,M10416****,4.8,家庭亲子,2017年08月,自主双床房,周围环境一般的，主要是在步行街上，去南京路方便了，地铁二号线乘坐也方便，贵州路上有1个安徽帅...,"[方便, 南京路, 卫生间, 环境, 步行, 房间]"
4,7天,步行街,1500108****,3.3,朋友出游,2017年07月,自主大床房,位置可以，就是酒店服务太垃圾，前台告知只有电卡没有房卡，开房门得服务员跟着，还让交200押金...,"[服务, 房间, 位置, 前台]"
5,7天,步行街,浅蓝色40,3.8,情侣出游,2017年06月,自主大床房,酒店还可以，感觉挺破了，卫生也是一般，如果特别爱干净的朋友就选择其他家吧，比如边上的汉庭酒店...,"[干净, 汉庭, 朋友]"
6,7天,步行街,差一点点04,4.5,家庭亲子,2017年05月,商务大床房,大上海真是寸土寸金啊，尤其是紧邻南京东路步行街 。酒店位置很好，在步行街第一商店旁边，对过就...,"[交通, 步行, 位置, 方便]"
7,7天,步行街,1395103****,3.5,家庭亲子,2017年07月,自主双床房,地理位置没话说，去哪都方便，房间设施确实太陈旧，电话、电视全是坏的，电吹风要去楼道电梯口才能...,"[方便, 隔音, 电话, 位置, 设施, 房间]"
8,7天,步行街,E0582****,3.0,家庭亲子,2017年08月,自主双床房,酒店位置相当好，在步行街，第一食品店侧面，晚上步行去外滩，距离正好。房间状况太差，迄今为止最...,"[南京路, 位置, 晚上, 外滩, 步行, 房间]"
9,7天,步行街,e04450687,1.0,代人预订,2017年06月,自主大床房,酒店在一条小街上，唯一好处，离步行街进，但凡有点别的要求的请不要订这个酒店。一进门屋里一股味...,"[朋友, 空调, 电话, 前台, 入住, 晚上, 步行, 房间]"


In [91]:
data.to_csv("clean_xiecheng_keywords_0920.txt",sep = "\t",encoding = "utf-8",index = False)

# Construct pandas DataFrame from items in nested dictionary

In [92]:
user_dict = {12: {'Category 1': {'att_1': 1, 'att_2': 'whatever'},
                  'Category 2': {'att_1': 23, 'att_2': 'another'}},
             15: {'Category 1': {'att_1': 10, 'att_2': 'foo'},
                  'Category 2': {'att_1': 30, 'att_2': 'bar'}}}

In [93]:
pd.DataFrame.from_dict({(i,j): user_dict[i][j] 
                           for i in user_dict.keys() 
                           for j in user_dict[i].keys()},
                       orient='index')

att_1     att_2
12 Category 1      1  whatever
   Category 2     23   another
15 Category 1     10       foo
   Category 2     30       bar

In [94]:
from pandas.io.json import json_normalize

In [96]:
data=[dict(user=dict(uid=123,full_name='Alice'),followers=1),dict(user=dict(uid=456,full_name='Bob'),followers=2)]
data

[{'followers': 1, 'user': {'full_name': 'Alice', 'uid': 123}},
 {'followers': 2, 'user': {'full_name': 'Bob', 'uid': 456}}]

In [97]:
json_normalize(data)

,followers,user.full_name,user.uid
0,1,Alice,123
1,2,Bob,456
